In [37]:
import pandas as pd
import numpy as np
import re
from PIL import Image
import pytesseract
import matplotlib.pyplot as plt
import cv2
import os,glob

# Writing the ocr strings to seperate text files

In [63]:
'''path = '../data/img'
tempPath ="../data/text"
for imageName in os.listdir(path):
    inputPath = os.path.join(path, imageName)
    img = cv2.imread(inputPath)
    img2 = cv2.resize(img ,None,fx=1.5,fy=1.5)
    gray = cv2.cvtColor(img2,cv2.COLOR_BGR2GRAY)
    adaptive = cv2.adaptiveThreshold(gray,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,87,11)

    text = pytesseract.image_to_string(adaptive)

    #imagePath = imagePath[0:-4]

    fullTempPath = os.path.join(tempPath, imageName+".txt")

    # saving the  text for every image in a separate .txt file
    file1 = open(fullTempPath, "w")
    file1.write(text)
    file1.close() '''

# Reading the text files and create the features dataframe

In [38]:
receipt_ocr = {}
date_lst=[]
time_lst=[]

In [45]:
time_lst

['8:13:39',
 '20:49:59',
 None,
 None,
 None,
 '8:01:11',
 '09:44:00',
 '13:14:15',
 None,
 '10:27:03',
 None,
 '15:29:00',
 None,
 '10:17:32',
 None,
 None,
 '07:10:35',
 '03:55:07',
 '07:10:35',
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 '5:23:26',
 None,
 None,
 None,
 None,
 '08:48:00',
 None,
 None,
 '6:41:08',
 None,
 None,
 '10:25:49',
 '10:58:51',
 None,
 '23:03:06',
 None,
 None,
 None,
 None,
 '12:49:00',
 '17:24:00',
 '18:43:34',
 None,
 '9:34:40',
 None,
 '12:42:00',
 None,
 None,
 None,
 None,
 '09:42:19',
 '20:29:59',
 '18:32:36',
 None,
 None,
 None,
 '20:54:50',
 None,
 '12:17:26',
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 '18:31:45',
 None,
 None,
 '10:20:37',
 None,
 '3:43:01',
 '10:18:02',
 None,
 None,
 None,
 None,
 None,
 None,
 '03:00:42',
 '03:30:27',
 '3:13:19',
 None,
 None,
 None,
 None,
 None,
 None,
 '16:15:00',
 None,
 '08:44:00',
 None,
 None,
 '18:40:97',
 '13:62:42',
 '16:37:00',
 '13:29:51',
 None,
 Non

In [39]:
txtFilePath ="../data/text"
for fileName in os.listdir(txtFilePath):
    FilePath = os.path.join(txtFilePath, fileName)
    f=open(FilePath,'r')
    k=''
    for line in f:
        l=line
        k+=l

    date_pattern = r'(0[1-9]|[12][0-9]|3[01])([/]|[-])(0[1-9]|1[012])([/]|[-])(19|20)\d\d'
    date = re.search(date_pattern, k)
    if type(date) != type(None):
        date =  date.group()
    date_lst.append(date)

    time_pattern = r'(\d\d|\d)[:](\d\d)[:](\d\d)'
    time = re.search(time_pattern, k)
    if type(time) != type(None):
        time =  time.group()
    time_lst.append(time)


In [40]:
receipt_ocr['date'] = date_lst
receipt_ocr['time'] = time_lst

In [41]:
df_features = pd.DataFrame(receipt_ocr)

In [42]:
df_features

,date,time
0,25/12/2018,8:13:39
1,19/10/2016,20:49:59
2,None,None
3,25/12/2018,None
4,None,None
...,...,...
596,05-06-2018,20:23:14
597,22-05-2018,19:54:52
598,28-05-2018,23:28:57
599,19-05-2018,12:46:58


In [9]:
import pandas as pd
results = pd.read_json (r'results-all.json')
print (results)

    file_name   total
0    000.json    9.00
1    001.json   60.30
2    002.json   33.90
3    003.json   80.90
4    004.json   30.90
..        ...     ...
596  596.json   52.00
597  597.json   71.00
598  598.json  112.35
599  599.json   42.40
600  600.json    4.70

[601 rows x 2 columns]


In [10]:
# check missing values

num_of_null_columns = len([col for col in df_features.columns if sum(df_features[col].isnull())])
print(num_of_null_columns)

2


In [11]:
# check for duplications

df_features_len_before = len(df_features)
df_features.drop_duplicates()
df_features_len_after = len(df_features)
print(df_features_len_before-df_features_len_after)

0


# Split the data into training and validation

In [12]:
X = df_features.copy()
y = results['total']

In [13]:
from sklearn.model_selection import train_test_split
X_train,X_val,y_train,y_val = train_test_split(X,y,train_size=0.8,random_state=0)

In [50]:
from sklearn.impute import SimpleImputer
import pandas as pd
# Imputation
my_imputer = SimpleImputer(strategy='constant',fill_value='18/09/2018')
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
imputed_X_val = pd.DataFrame(my_imputer.transform(X_val))

# Imputation removed column names; put them back
imputed_X_train.columns = X_train.columns
imputed_X_val.columns = X_val.columns

In [51]:
imputed_X_train

,date,time
0,None,None
1,None,None
2,None,None
3,02/01/2018,09:28:00
4,14-03-2018,10:22:59
...,...,...
475,None,03:17:50
476,18/01/2018,10:27:03
477,20/10/2017,None
478,None,None


In [33]:

num_of_null_columns2 = len([col for col in imputed_X_train.columns if sum(imputed_X_train[col].isnull())])
print(num_of_null_columns2)

2
